In [1]:
!pip install -U langchain langchain-community langchain-openai sentence-transformers faiss-cpu gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [2]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from google.colab import drive
from google.colab import userdata
from pathlib import Path
from tqdm.notebook import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
transcript_folder = Path("/content/drive/MyDrive/ServiceNow_Audio_Transcripts")
transcript_folder.mkdir(parents=True, exist_ok=True)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

documents: list[Document] = []
for txt_path in tqdm(transcript_folder.glob("*.txt"), desc="Chunking transcripts"):
    raw_text = txt_path.read_text(encoding="utf-8")
    for chunk in splitter.split_text(raw_text):
        documents.append(
            Document(page_content=chunk, metadata={"source": txt_path.name})
        )
print(f"✅ Loaded and chunked {len(documents)} document chunks from {transcript_folder}.")

Chunking transcripts: 0it [00:00, ?it/s]

✅ Loaded and chunked 656 document chunks from /content/drive/MyDrive/ServiceNow_Audio_Transcripts.


In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_dir = "/content/drive/MyDrive/faiss_store"
faiss_dir = Path(faiss_dir)
Path(faiss_dir).mkdir(parents=True, exist_ok=True)

faiss_index = FAISS.load_local(
    faiss_dir,
    embedding_model,
    allow_dangerous_deserialization=True
)
retriever = faiss_index.as_retriever(search_kwargs={"k": 5})

/tmp/ipython-input-5-1688787792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
llm = OpenAI(
    temperature=0,
    openai_api_key=userdata.get("OPENAI_API_KEY")
)

retriever = faiss_index.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [7]:
from typing import Tuple, Union
import gradio as gr
from transformers import pipeline

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-small")

def answer_question(text_input: str, audio_input: Union[str, dict, None]) -> Tuple[str, str]:
    """
    Runs the RetrievalQA chain and formats the answer along with source snippets.
    Handles both text and audio input.

    Args:
        text_input: The string input from the text box.
        audio_input: The audio input, expected as a string filepath from Gradio.

    Returns:
      - answer text
      - formatted source list (one entry per document)
    """
    try:
        question = ""
        print(f"Debug: text_input type: {type(text_input)}, value: '{text_input}'")
        print(f"Debug: audio_input type: {type(audio_input)}, value: {audio_input}")

        if isinstance(audio_input, str) and audio_input:
             audio_filepath = audio_input
             print(f"Debug: Processing audio from filepath: {audio_filepath}")
             try:
                 transcription_result = transcriber(audio_filepath)
                 print(f"Debug: Transcription result: {transcription_result}")
                 question = transcription_result.get("text", "")
                 if not question:
                     print("Debug: Transcription resulted in empty string.")
                     return "Could not transcribe audio. Please try again.", ""
                 question = question.strip()
                 if not question:
                     print("Debug: Stripped transcription is empty.")
                     return "Transcribed audio was empty. Please try again.", ""
             except Exception as trans_e:
                 print(f"Debug: Error during transcription: {trans_e}")
                 return f"Error during transcription: {trans_e}", ""

        if not question and isinstance(text_input, str) and text_input.strip():
            print("Debug: Using text input.")
            question = text_input.strip()
            if not question:
                print("Debug: Stripped text input is empty.")
                return "Please enter a question or record audio.", ""

        if not question:
             print("Debug: Final question is empty after checking both inputs.")
             return "No valid question provided via text or audio.", ""

        print(f"Debug: Final question for QA chain: '{question}'")

        try:
            print("Debug: Calling qa_chain...")
            response = qa_chain(question)
            print(f"Debug: qa_chain response type: {type(response)}")
            print(f"Debug: qa_chain response keys: {response.keys() if isinstance(response, dict) else 'Not a dict'}")
            answer = response.get("result", "")
            print(f"Debug: Answer extracted: '{answer}')")

            sources = []
            source_documents = response.get("source_documents", [])
            print(f"Debug: Number of source documents: {len(source_documents)}")
            for doc in source_documents:
                text = doc.page_content.strip()
                preview = text[:400] + "..." if len(text) > 400 else text
                sources.append(f"📄 {doc.metadata.get('source', 'unknown')}\n🔎 {preview}")
            print(f"Debug: Sources formatted. Number of formatted sources: {len(sources)}")

            if not answer:
                 print("Debug: Answer is empty after qa_chain.")
                 return "Could not find a relevant answer in the documents.", "\n\n".join(sources)


            return answer, "\n\n".join(sources)

        except Exception as qa_e:
            print(f"Debug: Error during qa_chain execution or response processing: {qa_e}")
            return f"Error processing question with QA chain: {qa_e}", ""


    except Exception as e:
        err = f"An unexpected error occurred in answer_question: {e}"
        print(f"Debug: Unexpected error in answer_question: {e}")
        return err, err

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [8]:
code = '''import os
from typing import Tuple, Union
import gradio as gr
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from pathlib import Path
from transformers import pipeline
from google.colab import userdata # Assuming you'll run this part in Colab to get the key

# --- Configuration ---
# You might need to securely load this in a production app (e.g., using environment variables)
# For Colab, you can get it from userdata secrets
openai_api_key = userdata.get("OPENAI_API_KEY")
if not openai_api_key:
    raise EnvironmentError("OPENAI_API_KEY not set. Please add it to Colab secrets.")

# Define the path to your saved FAISS index
faiss_store = "/content/drive/MyDrive/faiss_store"
if not Path(faiss_store).exists():
     raise FileNotFoundError(f"FAISS index not found at {faiss_store}. Please ensure it's saved there.")

# --- Model and Chain Initialization ---

# Load the speech-to-text model
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Initialize the embedding model (used for loading FAISS)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Load the FAISS index from the local directory
faiss_index = FAISS.load_local(
    faiss_store,
    embedding_model,
    allow_dangerous_deserialization=True # Be cautious with untrusted sources
)

# Initialize the OpenAI LLM
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

# Create the RetrievalQA chain
retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# --- Gradio Interface Function ---

def answer_question(text_input: str, audio_input: Union[str, dict, None]) -> Tuple[str, str]:
    """
    Runs the RetrievalQA chain and formats the answer along with source snippets.
    Handles both text and audio input.

    Args:
        text_input: The string input from the text box.
        audio_input: The audio input, expected as a string filepath from Gradio.

    Returns:
      - answer text
      - formatted source list (one entry per document)
    """
    try:
        question = ""

        # Process audio input if provided (handle as a string filepath)
        if isinstance(audio_input, str) and audio_input:
             audio_filepath = audio_input
             try:
                 transcription_result = transcriber(audio_filepath)
                 question = transcription_result.get("text", "")
                 if not question:
                     return "Could not transcribe audio. Please try again.", ""
                 question = question.strip() # Remove leading/trailing whitespace from transcription
                 if not question:
                     return "Transcribed audio was empty. Please try again.", ""
             except Exception as trans_e:
                 return f"Error during transcription: {trans_e}", ""


        # Use text input if audio was not provided or transcription failed
        if not question and isinstance(text_input, str) and text_input.strip():
            question = text_input.strip()
            if not question:
                return "Please enter a question or record audio.", ""

        # Ensure there is a question to process after handling input types
        if not question:
             return "No valid question provided via text or audio.", ""

        # Run the QA chain with the processed question
        try:
            response = qa_chain(question)
            answer = response.get("result", "") # Use .get() with a default value

            # Build source list with preview snippets
            sources = []
            source_documents = response.get("source_documents", []) # Use .get() with a default empty list
            for doc in source_documents:
                text = doc.page_content.strip()
                preview = text[:400] + "..." if len(text) > 400 else text
                sources.append(f"📄 {doc.metadata.get('source', 'unknown')}\n🔎 {preview}")

            if not answer:
                 # It's possible the QA chain returned successfully but with no relevant answer
                 return "Could not find a relevant answer in the documents.", "\n\n".join(sources)


            return answer, "\n\n".join(sources)

        except Exception as qa_e:
            return f"Error processing question with QA chain: {qa_e}", ""


    except Exception as e:
        err = f"An unexpected error occurred in answer_question: {e}"
        # Return the same error in both slots to display in UI
        return err, err

# --- Gradio Interface Definition ---

gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(lines=2, placeholder="Ask a ServiceNow question via text...", label="Text Input"), # Text input
        gr.Audio(sources=["microphone"], type="filepath", label="Ask your question via microphone") # Audio input
    ],
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Sources")
    ],
    title="ServiceNow QA Assistant",
    description="RAG-powered assistant over your ServiceNow YouTube transcripts with text and speech input.",
    allow_flagging="never" # Use flagging_mode instead in newer Gradio versions
).launch()
'''

with open("app.py", "w") as f:
    f.write(code)

In [9]:
requirements = '''gradio
langchain
langchain-openai
langchain-community
langchain-core
sentence-transformers
transformers
faiss-cpu
openai
yt-dlp
python-dotenv
'''
with open("requirements.txt", "w") as f:
    f.write(requirements)

In [10]:
from google.colab import files
files.download("app.py")
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import gradio as gr

def process_audio_with_agent(audio_file):
    text_query = transcribe_audio(audio_file)
    answer = agent.run(input=text_query)
    return answer

def process_text_with_agent(text_query):
    answer = agent.run(input=text_query)
    return answer

def chat_fn(user_text, audio, history):
    if user_text and user_text.strip() != "":
        answer = process_text_with_agent(user_text)
        history = history + [[user_text, answer]]
    elif audio:
        answer = process_audio_with_agent(audio)
        history = history + [[None, answer]]
    return history, history

with gr.Blocks() as demo:
    gr.HTML("<h1 style='text-align: center;'>YouTube RAG ChatBot</h1>")
    gr.Markdown("<center>Type or record your question below. The bot will answer just like ChatGPT!</center>")
    chatbot = gr.Chatbot(label="Chat")
    state = gr.State([])
    with gr.Row():
        text_input = gr.Textbox(
            show_label=False,
            placeholder="Type your message...",
            scale=4,
            container=False,
        )
        audio_input = gr.Audio(
            type="filepath",
            label="🎤",
            scale=1,
            container=False,
        )
        send_btn = gr.Button("Send", scale=1)
    send_btn.click(
        chat_fn,
        inputs=[text_input, audio_input, state],
        outputs=[chatbot, state],
        preprocess=False,
    )
    text_input.submit(
        chat_fn,
        inputs=[text_input, audio_input, state],
        outputs=[chatbot, state],
        preprocess=False,
    )
if __name__ == "__main__":
    demo.launch()


/tmp/ipython-input-12-1775149583.py:24: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ca458f38b4ad130e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
